# 🤖 HuiHui-Qwen3-4B для AI Orchestrator

Этот блокнот настраивает модель HuiHui-Qwen3-4B-Thinking для работы с AI Orchestrator через удаленный API.

## Возможности
- ⚡ GPU ускорение на Google Colab
- 🧠 Квантованная модель для экономии памяти
- 🌐 API сервер для удаленного доступа
- 🔗 Динамическое получение системного промпта через API

In [ ]:
# 🧠 Системный промпт (будет получен динамически)
SYSTEM_PROMPT = ''

def update_system_prompt(new_prompt):
    """Обновляет системный промпт из внешнего источника"""
    global SYSTEM_PROMPT
    SYSTEM_PROMPT = new_prompt
    print(f'🔄 Системный промпт обновлен: {len(SYSTEM_PROMPT)} символов')

print('📝 Готов к получению системного промпта через API')
print(f'📏 Текущий промпт: {len(SYSTEM_PROMPT)} символов')

In [ ]:
# Установка зависимостей для HuiHui-Qwen3
!pip install transformers torch accelerate bitsandbytes
!pip install flask ngrok-python requests

import torch
import json
import time
import requests
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from flask import Flask, request, jsonify
import threading

print(f'🔥 GPU доступно: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'🖥️ GPU устройство: {torch.cuda.get_device_name(0)}')
    print(f'💾 GPU память: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB')
else:
    print('⚠️ GPU недоступно, используется CPU')

In [ ]:
# Загрузка HuiHui-Qwen3-4B модели
model_name = "HuiHui/Huihui-Qwen3-4B-Thinking-2507-abliterated"

print(f'📥 Загружаем модель: {model_name}')
print('⏳ Это может занять несколько минут...')

# Конфигурация для квантования (экономия памяти)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# Загружаем токенизатор
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

# Добавляем pad_token если его нет
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Загружаем модель с квантованием
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

print('✅ Модель HuiHui-Qwen3 загружена успешно!')
print(f'📊 Параметры модели: ~4B')
print(f'💾 Используемая память: {torch.cuda.memory_allocated()/1024**3:.1f} GB')

In [ ]:
# API функции для обработки запросов
def generate_response(prompt, max_tokens=2048, temperature=0.7, use_system_prompt=True):
    """Генерирует ответ используя HuiHui-Qwen3 модель"""
    try:
        # Формируем полный промпт с глобальным системным промптом
        if use_system_prompt and SYSTEM_PROMPT:
            full_prompt = f"{SYSTEM_PROMPT}\n\nПользователь: {prompt}\n\nАссистент:"
        else:
            full_prompt = f"Human: {prompt}\n\nAssistant:"
        
        # Токенизация
        inputs = tokenizer(
            full_prompt, 
            return_tensors="pt", 
            truncation=True, 
            max_length=4096
        ).to(model.device)
        
        # Генерация
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        
        # Декодирование ответа
        response = tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:], 
            skip_special_tokens=True
        ).strip()
        
        return {
            'status': 'success',
            'response': response,
            'tokens_used': outputs[0].shape[1],
            'model': 'huihui-qwen3-4b-colab',
            'system_prompt_used': use_system_prompt and bool(SYSTEM_PROMPT)
        }
        
    except Exception as e:
        return {
            'status': 'error',
            'error': str(e),
            'model': 'huihui-qwen3-4b-colab'
        }

print('✅ API функции готовы!')

In [ ]:
# Flask API сервер для приема запросов
app = Flask(__name__)

@app.route('/health', methods=['GET'])
def health_check():
    """Проверка состояния сервера"""
    return jsonify({
        'status': 'healthy',
        'model': 'huihui-qwen3-4b-colab',
        'gpu_available': torch.cuda.is_available(),
        'memory_used': f"{torch.cuda.memory_allocated()/1024**3:.1f} GB" if torch.cuda.is_available() else "N/A",
        'system_prompt_loaded': bool(SYSTEM_PROMPT),
        'system_prompt_length': len(SYSTEM_PROMPT)
    })

@app.route('/update_prompt', methods=['POST'])
def update_prompt():
    """Обновляет системный промпт"""
    try:
        data = request.get_json()
        new_prompt = data.get('system_prompt', '')
        
        update_system_prompt(new_prompt)
        
        return jsonify({
            'status': 'success',
            'message': 'Системный промпт обновлен',
            'prompt_length': len(SYSTEM_PROMPT)
        })
        
    except Exception as e:
        return jsonify({
            'status': 'error',
            'error': str(e)
        }), 500

@app.route('/generate', methods=['POST'])
def generate_text():
    """Основной endpoint для генерации текста"""
    try:
        data = request.get_json()
        
        prompt = data.get('prompt', '')
        max_tokens = data.get('max_tokens', 2048)
        temperature = data.get('temperature', 0.7)
        use_system_prompt = data.get('use_system_prompt', True)
        
        # Если в запросе есть системный промпт, обновляем его
        if 'system_prompt' in data and data['system_prompt']:
            update_system_prompt(data['system_prompt'])
        
        if not prompt:
            return jsonify({'status': 'error', 'error': 'Prompt is required'}), 400
        
        # Генерируем ответ
        result = generate_response(
            prompt=prompt, 
            max_tokens=max_tokens, 
            temperature=temperature, 
            use_system_prompt=use_system_prompt
        )
        
        return jsonify(result)
        
    except Exception as e:
        return jsonify({
            'status': 'error',
            'error': str(e)
        }), 500

print('🌐 Flask API сервер настроен!')
print('📡 Доступные endpoints:')
print('  GET  /health - проверка состояния')
print('  POST /update_prompt - обновление системного промпта')
print('  POST /generate - генерация текста')

In [ ]:
# Настройка ngrok туннеля для внешнего доступа
from pyngrok import ngrok
import threading

# Получаем токен ngrok (если есть)
# ngrok.set_auth_token("YOUR_NGROK_TOKEN")  # Раскомментируйте и добавьте токен

# Запускаем Flask в отдельном потоке
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Даем время Flask запуститься
time.sleep(3)

# Создаем ngrok туннель
public_url = ngrok.connect(5000)
print(f'🚀 Сервер запущен!')
print(f'🔗 Публичный URL: {public_url}')
print(f'🔗 Health check: {public_url}/health')
print(f'🔗 Update prompt: {public_url}/update_prompt')
print(f'🔗 Generate endpoint: {public_url}/generate')

# Сохраняем URL для подключения
with open('colab_server_url.txt', 'w') as f:
    f.write(str(public_url))

print('\n✅ HuiHui-Qwen3 модель готова к приему запросов от AI Orchestrator!')
print('📋 Скопируйте публичный URL и используйте его в плагине для подключения.')
print(f'🧠 Готов к получению системного промпта через: {public_url}/update_prompt')

In [ ]:
# Тестирование endpoints
import time
time.sleep(2)  # Даем время серверу запуститься

# Получаем URL из переменной ngrok
try:
    base_url = str(public_url)
    
    # Тест health check
    print("🔍 Тестируем health check...")
    health_response = requests.get(f"{base_url}/health")
    if health_response.status_code == 200:
        health_data = health_response.json()
        print(f"✅ Health check успешен: {health_data['status']}")
        print(f"📊 GPU доступно: {health_data['gpu_available']}")
        print(f"💾 Память: {health_data['memory_used']}")
    else:
        print(f"❌ Health check неудачен: {health_response.status_code}")
    
    # Тест обновления промпта
    print("\n🔍 Тестируем обновление системного промпта...")
    test_prompt = "Ты полезный AI ассистент, работающий в Google Colab."
    prompt_response = requests.post(f"{base_url}/update_prompt", 
                                  json={"system_prompt": test_prompt})
    if prompt_response.status_code == 200:
        prompt_data = prompt_response.json()
        print(f"✅ Промпт обновлен: {prompt_data['prompt_length']} символов")
    else:
        print(f"❌ Ошибка обновления промпта: {prompt_response.status_code}")
    
    # Тест генерации (небольшой)
    print("\n🔍 Тестируем генерацию...")
    gen_response = requests.post(f"{base_url}/generate", 
                               json={
                                   "prompt": "Привет!", 
                                   "max_tokens": 50,
                                   "temperature": 0.7
                               })
    if gen_response.status_code == 200:
        gen_data = gen_response.json()
        if gen_data['status'] == 'success':
            print(f"✅ Генерация успешна: {gen_data['response'][:50]}...")
            print(f"🧠 Системный промпт использован: {gen_data['system_prompt_used']}")
        else:
            print(f"❌ Ошибка генерации: {gen_data['error']}")
    else:
        print(f"❌ Ошибка запроса генерации: {gen_response.status_code}")
        
except Exception as e:
    print(f"❌ Ошибка тестирования: {e}")

print("\n🎯 Тестирование завершено!")

## 🎯 Готов к работе!

Модель HuiHui-Qwen3-4B загружена и API сервер запущен.

**Как использовать:**
1. Скопируйте публичный URL из предыдущих ячеек
2. Используйте этот URL в AI Orchestrator плагине
3. Системный промпт передается динамически через API

**Пример обновления системного промпта:**
```json
POST /update_prompt
{
  "system_prompt": "Твой новый системный промпт..."
}
```

**Пример запроса с промптом:**
```json
POST /generate
{
  "prompt": "Расскажи о искусственном интеллекте",
  "max_tokens": 512,
  "temperature": 0.7,
  "system_prompt": "Промпт можно передать и здесь",
  "use_system_prompt": true
}
```

**Преимущества динамического промпта:**
- ✅ Можно обновлять промпт без перезапуска модели
- ✅ Передача через защищенный ngrok туннель
- ✅ Отдельный endpoint для управления промптом
- ✅ Проверка статуса через /health endpoint